## 1. 필요한 라이브러리 설치

In [1]:
pip install numpy pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install trl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install -U transformers
# pip install -U accelerate

In [6]:
pip install langchain langchain-community pypdf pdfplumber faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
# pip install kiwipiepy

In [8]:
pip install langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install rank_bm25

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 2. RAG + LLM 모델 구상

In [10]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer

from langchain_community.document_loaders import PyPDFLoader # 1.로드
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

from transformers import pipeline

/home/minhai/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
# # 1. 문서 로드 (pdf페이지로 나눠짐)
# from tqdm import tqdm
# from glob import glob

# pdf_folder = "/home/minhai/2024-2-DSCD-Synnovation-3/보험자료"
# pdf_files = glob(os.path.join(pdf_folder, '*.pdf'))
# print(pdf_files)

# data = []
# for pdf_file in tqdm(pdf_files, desc="Reading PDF files"):
#     loader = PyPDFLoader(pdf_file)
#     data += loader.load()

# len(data)

In [12]:
# 1. 문서 로드 (pdf페이지로 나눠짐)
loader = PyPDFLoader("/home/minhai/2024-2-DSCD-Synnovation-3/한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관.pdf")
documents = loader.load()

In [13]:
# 2. 문서를 적절한 크기의 조각으로 청크 (split)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30) # 청크 중 중복되는 부분 크기
  #seprator 공백이면 공백 기준으로 청크를.. \n을 기본적으로 사용 (이거는 순차적으로 진행해줌)

chunks = text_splitter.split_documents(documents)
chunks = chunks[11:] #불필요한 청크 제외(목차 등)

# 각 청크에 id 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in chunks)}")

생성된 텍스트 청크 수: 1750
각 청크의 길이: [490, 144, 487, 119, 308, 474, 376, 220, 496, 182, 497, 479, 54, 285, 231, 498, 289, 491, 407, 495, 399, 490, 494, 37, 482, 34, 482, 393, 486, 446, 471, 275, 475, 91, 466, 481, 369, 493, 238, 460, 86, 481, 469, 197, 467, 478, 210, 474, 397, 471, 153, 280, 453, 482, 480, 229, 479, 457, 484, 431, 232, 492, 463, 452, 472, 153, 487, 482, 483, 493, 48, 459, 448, 461, 309, 448, 464, 457, 392, 451, 495, 495, 424, 484, 465, 477, 484, 104, 489, 487, 453, 395, 497, 464, 498, 263, 486, 445, 482, 341, 458, 496, 482, 437, 487, 478, 463, 419, 484, 495, 467, 364, 490, 472, 487, 211, 473, 443, 495, 91, 483, 448, 190, 453, 488, 420, 498, 344, 472, 494, 222, 480, 476, 446, 474, 469, 449, 492, 479, 364, 495, 487, 250, 490, 470, 490, 59, 448, 470, 494, 459, 238, 341, 244, 471, 213, 461, 453, 484, 148, 494, 478, 449, 231, 485, 482, 490, 455, 54, 494, 478, 432, 458, 476, 481, 191, 473, 454, 462, 227, 498, 474, 487, 51, 496, 488, 306, 466, 484, 477, 365, 490, 493, 376, 488, 492, 

In [14]:
# 3. HuggingFaceBgeEmbeddings 사용하여 벡터 임베딩 생성 (의미 검색)
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") #기본모델:all-MiniLM-L6-v2 # model_name="BAAI/bge-small-en-v1.5", "BAAI/bge-m3"지정 가능

# 3. FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=chunks,
                                   embedding=embedding)

/tmp/ipykernel_8710/1925248646.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") #기본모델:all-MiniLM-L6-v2 # model_name="BAAI/bge-small-en-v1.5", "BAAI/bge-m3"지정 가능


키워드 검색만의 한계점을 없애기 위해서 앙상블 모델 활용 (의미 검색 + 키워드 검색)
- openai > 허깅페이스 > ollama (모델 개수)
- 특히 한국어에서는 앞 2가지보다 성능이 떨어질 수 있음 (따라서 허깅페이스 모델 사용)

In [15]:
# 4. 질의 실행 및 top-k 문서 검색 (similarity_search보다 검색기 활용하는 게 좋음)
query = "보험금 지급이 제한될 수 있는 경우는 뭐가 있어?"
top_k = 3  # 원하는 top-k 문서 개수 설정

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
results = retriever.invoke(query)

# 결과 출력
print("Top-k Documents:")
for doc in results:
    print(doc.page_content)  # 각 문서의 내용 출력

Top-k Documents:
한도로 한다. 
⑥ 법 제32조제2항에 따른 보험금의 지급한도는 5천만원(이하 "보험금 지급한도"라 한다)으로 한다. 
이 경우 다음 각 호의 경우에는 각 호에서 정한 바에 따라 보험금 지급한도를 적용한다. 
1. 확정기여형퇴직연금제도등의 경우: 가입자별로 보험금 지급한도를 적용하되, 확정기여형퇴직연
금제도등에 따른 예금등 채권과 그 밖의 예금등 채권에 대하여 각각 보험금 지급한도를 적용 
2. 개인종합자산관리계좌의 경우: 계좌보유자별로 보험금 지급한도를 적용하되, 개인종합자산관리
계좌의 예금등 채권과 그 밖의 예금등 채권(확정기여형퇴직연금제도등에 따른 예금등 채권은 
제외한다)을 합산하여 보험금 지급한도를 적용 
 
 
○ 영유아보육법 
 
제10조(어린이집의 종류) 
어린이집의 종류는 다음 각 호와 같다. 
1. 국공립어린이집 : 국가나 지방자치단체가 설치·운영하는 어린이집 
2. 사회복지법인어린이집 : 「사회복지사업법」 에 따른 사회복지법인(이하 "사회복지법인"이라 한다)이
해당 보험금 지급사유가 발생한 이후부터 보장하지 않으며, 그 때부터 효력이 없습니다. 
④ 보험수익자와 회사가 제2-3조(보험금의 지급사유)에 대해 합의하지 못할 때는 보험수익자와 회사가 
함께 제3자를 정하고 그 제3자의 의견에 따를 수 있습니다. 제3자는 의료법 제3조(의료기관)에 규정한 
종합병원 소속 전문의 중에서 정하며, 보험금 지급사유 판정에 드는 의료비용은 회사가 전액 부담합니
다. 
 
제 2-6 조 장해지급률에 관한 세부규정 
이 특약은 해당사항이 없습니다. 
 
제 2-7 조 보험금을 지급하지 않는 사유 
회사는 다음 중 어느 한 가지로 보험금 지급사유가 발생한 때에는 보험금을 지급하지 않습니다. 
1. 피보험자가 고의로 자신을 해친 경우 
다만, 피보험자가 심신상실 등으로 자유로운 의사결정을 할 수 없는 상태에서 자신을 해침으로 인
하여 보험금 지급사유가 발생한 때에는 보험금을 지급합니다. 
2. 보험수익자가 고의로 피보험자를 해친 경

In [16]:
# FAISS 인덱스 타입 확인
print(vectorstore.index)

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7e1b5030db90> >


- 기본모델보다는 해당 모델("BAAI/bge-m3")이 잘 나옴
- 기본 인덱스(IndexFlatL2/IndexFlatIP)를 사용한 경우:정확한 KNN(k-Nearest Neighbors) 검색이 수행

In [17]:
# # 키워드 검색
# from kiwipiepy import Kiwi # 한국어 토크나이저를 사용하여 문장을 토큰화하는 함수

# def bm25_process_func(text):
#     kiwi_model = Kiwi()
#     return [t.form for t in kiwi_model.tokenize(text)]

# bm25_retriever = BM25Retriever.from_documents(chunks, preprocess_func=bm25_process_func)
# bm25_retriever.k = 3
# results = bm25_retriever.get_relevant_documents(query) # BM25Retriever를 사용하여 query에 맞는 문서 검색

# # 검색 결과 출력
# for i, doc in enumerate(results):
#     print(f"Rank: {i+1}, Content: {doc.page_content}")

시간이 10분 넘게 걸려서 포기 (그냥 진행)

In [18]:
# 키워드 검색
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 3
results = bm25_retriever.get_relevant_documents(query) # BM25Retriever를 사용하여 query에 맞는 문서 검색

# 검색 결과 출력
for i, doc in enumerate(results):
    print(f"Rank: {i+1}, Content: {doc.page_content}")

Rank: 1, Content: 당뇨병으로 통원치료를 받아 보험금을 청구 
▶ 보험회사는 보험계약이 해지됨과 동시에 보험금 지급이 어려움을 안내 
  
  
  
  
   
  [법률 지식] 
일반적으로 보험설계사는 독자적으로 보험회사를 대리하여 보험계약을 체결할 권한 
이나 고지의무를 수령할 권한이 없음(대법원 2007. 6. 28. 선고 2006다59837) 
  
  
   
   
(5) 보험계약 후 
알릴의무 
위반 시 불이익 
해당 없음 
 보험계약자 등은 피보험자의 직업·직무 변경 등이 발생한 경우 지체없이 보험회사에 
 알려야 하며, 이를 위반하는 경우 보험금 지급이 제한될 수 있습니다. 
  
 보험회사는 피보험자의 직업·직무 변경 등으로 인해, 
  ① 위험이 감소한 경우 ▶ 보험료를 감액하고 정산금액을 환급하여 드립니다. 
  
② 위험이 증가한 경우 ▶ 보험료가 증액되고 정산금액의 추가 납입이 필요할 수 
                     있습니다.
Rank: 2, Content: 16 / 531 
 
2. 소비자가 반드시 알아야 할 유의사항 
(1) 보험금 지급이 제한될 수 있습니다. 
이 보험에는 면책기간, 감액지급, 보장한도와 자기부담금 등 보험금 지급제한 조건이 부가되어 있습니다. 
※ 자세한 내용 및 아래에 기재된 특약 이외의 부가특약의 내용은 본 상품의 약관 및 상품설명서를 참고 
   바랍니다. 
 
 
면책기간 
  
이 보험에는 ’보험금이 지급되지 않는 기간’을 의미하는 [면책기간]이 적용되는 담보가 
포함되어 있습니다. 
 
 구분 담보명 면책기간 
 
주계약 암수술자금 
암, 갑상선암 가입 후 90일간 보장제외 
 
기타피부암, 
대장점막내암, 
제자리암,경계성종양 
해당사항 없음 
 
간편가입 
암보장S특약 
[일반암] 
(갱신형) 
암진단자금 가입 후 90일간 보장제외 
 
간편가입 
부위별암보장S특약
[호흡기암(폐암 및  
후두암 포함)] 
(갱신형) 
호흡기암 진단자금 가입 후 90일간 보장제외 
  
간편

/tmp/ipykernel_8710/1769668125.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = bm25_retriever.get_relevant_documents(query) # BM25Retriever를 사용하여 query에 맞는 문서 검색


In [19]:
# 의미 검색 + 키워드 검색 (앙상블)
ensemble_retrievers = [retriever, bm25_retriever]
ensemble_retriever = EnsembleRetriever(
    retrievers=ensemble_retrievers,
    weights=[0.7, 0.3]
)

retrieved_docs = ensemble_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 보험금 지급이 제한될 수 있는 경우는 뭐가 있어?
검색 결과:
- 한도로 한다. 
⑥ 법 제32조제2항에 따른 보험금의 지급한도는 5천만원(이하 "보험금 지급한도"라 한다)으로 한다. 
이 경우 다음 각 호의 경우에는 각 호에서 정한 바에 따라 보험금 지급한도를 적용한다. 
1. 확정기여형퇴직연금제도등의 경우: 가입자별로 보험금 지급한도를 적용하되, 확정기여형퇴직연
금제도등에 따른 예금등 채권과 그 밖의 예금등 채권에 대하여 각각 보험금 지급한도를 적용 
2. 개인종합자산관리계좌의 경우: 계좌보유자별로 보험금 지급한도를 적용하되, 개인종합자산관리
계좌의 예금등 채권과 그 밖의 예금등 채권(확정기여형퇴직연금제도등에 따른 예금등 채권은 
제외한다)을 합산하여 보험금 지급한도를 적용 
 
 
○ 영유아보육법 
 
제10조(어린이집의 종류) 
어린이집의 종류는 다음 각 호와 같다. 
1. 국공립어린이집 : 국가나 지방자치단체가 설치·운영하는 어린이집 
2. 사회복지법인어린이집 : 「사회복지사업법」 에 따른 사회복지법인(이하 "사회복지법인"이라 한다)이 [출처: /home/minhai/2024-2-DSCD-Synnovation-3/한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관.pdf]
- 해당 보험금 지급사유가 발생한 이후부터 보장하지 않으며, 그 때부터 효력이 없습니다. 
④ 보험수익자와 회사가 제2-3조(보험금의 지급사유)에 대해 합의하지 못할 때는 보험수익자와 회사가 
함께 제3자를 정하고 그 제3자의 의견에 따를 수 있습니다. 제3자는 의료법 제3조(의료기관)에 규정한 
종합병원 소속 전문의 중에서 정하며, 보험금 지급사유 판정에 드는 의료비용은 회사가 전액 부담합니
다. 
 
제 2-6 조 장해지급률에 관한 세부규정 
이 특약은 해당사항이 없습니다. 
 
제 2-7 조 보험금을 지급하지 않는 사유 
회사는 다음 중 어느 한 가지로 보험금 지급사유가 발생한 때에는 보험금을 지급하지 않습니다. 
1. 피보험자가 고의로 자신을

키워드 검색으로 결정했지만, 해당 검색만으로는 한계가 있다고 생각해서
성능이 향상된 임베딩 모델로 교체 + 앙상블 모델로 구축

In [20]:
# # Ollama라는 특정 플랫폼이나 라이브러리에서 제공하는 모델 로드 방식

# 5. llm 모델 로드 (용태 beomi/Llama-3-Open-Ko-8B와 비슷한 모델인 듯, 이것도 8B)
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="llama3.1",
    temperature=0.8,
    num_predict=300, # 최대로 생성할 토큰 수
    base_url="http://localhost:11434")

response = llm.invoke("보험금 지급이 제한될 수 있는 경우는 뭐가 있어?")
print(f"LLM 응답: {response}")

LLM 응답: content='보험금 지급이 제한 될 수 있는 경우는 다음과 같습니다.\n\n1.  **위험성 제한**: 보험 계약의 내용에 따라 위험성이 너무 높거나 특정 활동을 하는 사람들이 보장받는 경우, 보험회사에서는 보험금 지급이 제한될 수 있습니다.\n2.  **보험 금지 사항:** 보험계약에서 지정된 특정 활동이나 상태(예를 들어, 의사 선고를 받은 사람)의 경우 보험금 지급이 거부 될 수 있습니다.\n3.  **보증 만기**: 보험 계약의 만기가 끝나거나 보험 Premiium 미지불로 인해 보험 계약이 종료되는 경우, 보험금 지급이 제한되거나 중단될 수 있습니다.\n4.  **보험 금지 시기:** 보험계약에서 지정된 특정 기간에만 보장이 이루어지는 경우, 보험금 지급은 해당 기간에만 유효합니다.\n\n보장 혜택을 받는 사람의 사소한 잘못이지만, 보험회사에서는 보험금 지급이 제한되거나 거부될 수 있습니다. 이 점에 대한 자세한 정보를 원한다면, 항목 5. "보험주의사항"에서 확인할 수 있습니다.' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2024-11-20T01:30:25.914660846Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 44355936599, 'load_duration': 1822129671, 'prompt_eval_count': 26, 'prompt_eval_duration': 1853428000, 'eval_count': 279, 'eval_duration': 40638155000} id='run-a9a8a83e-5467-4620-b449-825cedb1b086-0' usage_metadata={'input_tokens': 26, 'output_tokens': 279, 'total_tokens

In [ ]:
# 6. llm + rag 파이프라인 구성
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


# LLM 초기화
llm = ChatOllama(model="llama3.1",
                 temperature=0.8,
                 num_predict=500) # 최대로 생성할 토큰 수


# 프롬프트 작성
retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면,
“다시 보내기를 통해 답변을 다시 제공하겠습니다. 반복적으로 문제가 발생한다면 답변할 수 없는 내용입니다"라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")


# 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(ensemble_retriever, combine_docs_chain)

# 체인 실행
query = "보험금 지급이 제한될 수 있는 경우는 뭐가 있어?"
response = rag_chain.invoke({"input": query})
print(response)

{'input': '보험금 지급이 제한될 수 있는 경우는 뭐가 있어?', 'context': [Document(metadata={'source': '/home/minhai/2024-2-DSCD-Synnovation-3/한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관.pdf', 'page': 513, 'doc_id': 1684}, page_content='한도로 한다. \n⑥ 법 제32조제2항에 따른 보험금의 지급한도는 5천만원(이하 "보험금 지급한도"라 한다)으로 한다. \n이 경우 다음 각 호의 경우에는 각 호에서 정한 바에 따라 보험금 지급한도를 적용한다. \n1. 확정기여형퇴직연금제도등의 경우: 가입자별로 보험금 지급한도를 적용하되, 확정기여형퇴직연\n금제도등에 따른 예금등 채권과 그 밖의 예금등 채권에 대하여 각각 보험금 지급한도를 적용 \n2. 개인종합자산관리계좌의 경우: 계좌보유자별로 보험금 지급한도를 적용하되, 개인종합자산관리\n계좌의 예금등 채권과 그 밖의 예금등 채권(확정기여형퇴직연금제도등에 따른 예금등 채권은 \n제외한다)을 합산하여 보험금 지급한도를 적용 \n \n \n○ 영유아보육법 \n \n제10조(어린이집의 종류) \n어린이집의 종류는 다음 각 호와 같다. \n1. 국공립어린이집 : 국가나 지방자치단체가 설치·운영하는 어린이집 \n2. 사회복지법인어린이집 : 「사회복지사업법」 에 따른 사회복지법인(이하 "사회복지법인"이라 한다)이'), Document(metadata={'source': '/home/minhai/2024-2-DSCD-Synnovation-3/한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관.pdf', 'page': 262, 'doc_id': 833}, page_content='해당 보험금 지급사유가 발생한 이후부터 보장하지 않으며, 그 때부터 효력이 없습니다. \n④ 보험수익자와 회사가 제2-3조(보험금의 지급사유)에 대해 합의하지 못할 때는 보험수익자와 회사가

In [22]:
# 응답 객체의 키 확인
response.keys()

dict_keys(['input', 'context', 'answer'])

In [23]:
response["answer"]

'보험금 지급이 제한될 수 있는 경우는 다음과 같습니다.\n\n1. 고의 또는 중대한 과실로 인한 범죄행위에 그 원인이 있거나 고의로 사고를 일으킨 경우\n2. 고의 또는 중대한 과실로 공단이나 요양기관의 요양에 관한 지시에 따르지 아니한 경우\n3. 고의 또는 중대한 과실로 제55조에 따른 문서와 그 밖의 물건의 제출을 거부하거나 질문 또는 진단을 기피한 경우\n4. 업무 또는 공무로 생긴 질병ㆍ부상ㆍ재해로 다른 법령에 따른 보험급여나 보상(報償) 또는 보상(補償)을 받게 되는 경우\n5. 다른 법령에 따라 국가나 지방자치단체로부터 보험급여에 상당하는 급여를 받거나 보험급여에 상당하는 비용을 지급받게 되는 경우'

## 3. 답변 성능 평가
- Embedding Distance (ex.BERTScore)
- Cross-Encoder
- Rouge Metric -> BERT로도 충분하게 사용가능

> BERTScore로 (embedding distance, rouge metric)

### Cross-Encoder

In [62]:
# QA 데이터셋 로드 (200개 항목)
df_qa_test = pd.read_csv("/home/minhai/2024-2-DSCD-Synnovation-3/qa_test_최종.csv", encoding='cp949')
print(df_qa_test.shape)
df_qa_test.head()

(200, 5)


,context,source,doc_id,question,answer
0,7 / 531 \nI. 약관 이용 안내 \n1. 보험약관이란? \n보험약관은 가입하...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,0,약관은 어떤 구성으로 이루어져 있나요?,약관은 주계약 약관과 특약(특별약관)으로 나뉩니다. 주계약 약관은 기본 계약을 포함...
1,10 / 531 \n \n5. 보험금 신청방법 \n \nSTEP 1. \n보험...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,5,보험금을 신청하려면 어떤 서류가 필요한가요?,보험금 신청 시 필요한 서류는 신청하는 보험금 종류에 따라 다릅니다. 기본적으로는 ...
2,있을 수 있으니 청구 유형별로 세부내역을 확인하시기 바랍니다. \n 구분 사망 장...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,6,보험금 지급 절차는 어떻게 이루어지나요?,보험금 지급 절차는 다음과 같습니다:\n\nSTEP 1: 보험금 지급 접수\nSTE...
3,12 / 531 \nⅡ. 약관 요약서 \n1. 보험계약의 개요 \n가입하신 보험상품...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,8,해당 암보험은 어떤 특징을 가지고 있나요?,"이 보험 상품의 주요 특징은 다음과 같습니다: 보장성보험: 사망, 질병, 상해 등의..."
4,13 / 531 \n \n \n※ 3개월동안 대상특약의 보험료가 발생하지 않는 것...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,10,보장 개시일부터 보험료를 납입해야 하는 특약에는 어떤 것이 있나요?,보장 개시일부터 특약 보험료를 납입하는 특약으로는 다음이 포함됩니다:\n\n간편가입...


In [63]:
# 유사도 score를 return하는 함수 정의 
from sentence_transformers import CrossEncoder

def calculate_cross_encoder_similarity(
        query: str, 
        prediction: str, 
        model_name: str = "BAAI/bge-reranker-v2-m3", # "BAAI/bge-m3"
        ) -> float:
    """
    주어진 query와 prediction 사이의 의미적 유사성을 계산합니다.

    Args:
    query (str): 기준이 되는 쿼리 문장(정답 문장)
    prediction (str): 유사성을 비교할 예측 문장
    model_name (str): 사용할 크로스 인코더 모델 이름 (기본값: "BAAI/bge-reranker-v2-m3")

    Returns:
    float: 두 문장 간의 유사성 점수
    """
    # 크로스 인코더 모델
    cross_encoder_model = CrossEncoder(model_name)

    # 크로스 인코더를 사용하여 유사성 점수 계산
    sentence_pairs = [[query, prediction]]
    similarity_scores = cross_encoder_model.predict(sentence_pairs)

    return similarity_scores[0]

In [1]:
# 예시 (하나씩 확인)
question = df_qa_test.iloc[27]['question']
ground_truth = df_qa_test.iloc[27]['answer']
response = rag_chain.invoke({"input": question})
prediction = response['answer']

print("Question:", question) # 사용자 질문
print("Ground Truth:", ground_truth) # 진실
print("Prediction:", prediction) # 예측값 

similarity = calculate_cross_encoder_similarity(ground_truth, prediction)
print(f"유사성 점수: {similarity:.4f}")

NameError: name 'df_qa_test' is not defined

In [ ]:
# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 10개만 평가 (데이터프레임으로 정리)
def cross_encoder_evaluate_qa_dataset(df_qa_test):

    results = []

    for i in range(len(df_qa_test)):
        question = df_qa_test.iloc[i]['question']
        ground_truth = df_qa_test.iloc[i]['answer']
        response = rag_chain.invoke({"input": question})
        prediction = response['answer'] 
        
        similarity = calculate_cross_encoder_similarity(ground_truth, prediction)
        results.append({
            "index": df_qa_test.index[i],  # 원본 인덱스 유지
            "similarity": f"{similarity:.3f}", 
            "prediction": prediction
        })

    return pd.DataFrame(results)

df_result_cross_encoder = cross_encoder_evaluate_qa_dataset(df_qa_test.iloc[30:40])
df_result_cross_encoder

,index,similarity,prediction
0,20,1.000,보험계약이 무효가 되는 상황은 다음과 같습니다.\n\n① 타인의 사망을 보험금 지...
1,21,0.966,"계약 전 알릴의무 위반 시, 보험금 청구 시 불이익을 받을 수 있습니다. 만약 피보..."
2,22,1.000,"보험계약 전 알릴의무 위반 시, 보험계약이 해지 또는 보험금 지급이 어려울 수 있습니다."
3,23,1.000,"고의로 사실과 다르게 알린 경우, 보험회사는 계약을 해지하거나 보장을 제한할 수 있..."
4,24,1.000,"보험회사는 보험료 납입이 연체되었을 때 납입최고(독촉) 안내를 하며, 14일 이상(..."
5,25,0.989,보험계약대출을 받을 때 주의해야 할 사항으로는 다음과 같은 것이 있습니다.\n\n①...
6,26,0.947,STEP 1. 보험금 신청 전 서류를 준비해주세요.\n 구비서류는 홈페이지(www....
7,27,0.011,"이 보험의 목적은 질병을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으..."
8,28,0.998,"연단위 복리 계산 예시로, 원금 10,000원에 대한 1년간 2% 이자율이 적용된 ..."
9,29,0.831,"제9차 개정 한국표준질병∙사인분류에서 보장하는 질병에 해당된다고 하더라도, '대상이..."


In [28]:
# df_result_cross_encoder.to_csv("/home/minhai/2024-2-DSCD-Synnovation-3/cross_encoder_10.csv")

3개 돌렸는데 5분 걸리는데 어카지.... 200개 해야하는디 </br>
20개 83분...